In [1]:
import pandas as pd
import requests
import time
from datetime import datetime

In [3]:
try:
    df_deputados = pd.read_csv("Info deputados.csv")
    print(f"Carregado lista de {len(df_deputados)} deputados.")
except FileNotFoundError:
    print("Erro: Arquivo 'Info deputados.csv' não encontrado.")
    exit()

Carregado lista de 18 deputados.


In [4]:
# Configurações da API
base_url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"
headers = {"Accept": "application/json"}

# 2. Definir os intervalos de data (trimestrais) para cobrir o ano de 2025
# A API exige janelas curtas de tempo.
intervalos = [
    ("2025-01-01", "2025-03-31"),
    ("2025-04-01", "2025-06-30"),
    ("2025-07-01", "2025-09-30"),
    ("2025-10-01", "2025-12-31")
]

todas_proposicoes = []

for index, row in df_deputados.iterrows():
    nome_deputado = row['nome']
    print(f"Buscando: {nome_deputado} ({index + 1}/{len(df_deputados)})...")
    
    for data_inicio, data_fim in intervalos:
        params = {
            "siglaTipo": "PEC,PL,PLP,PLV",
            "ano": 2025,
            "autor": f'"{nome_deputado}"',
            "dataInicio": data_inicio,
            "dataFim": data_fim,
            "ordem": "ASC",
            "ordenarPor": "siglaTipo",
            "pagina": 1,
            "itens": 100
        }
        
        url_atual = base_url
        
        while True:
            try:
                response = requests.get(url_atual, params=params, headers=headers)
                
                # --- MUDANÇA 2: Tratamento Inteligente de Bloqueio (Erro 429) ---
                if response.status_code == 429:
                    print("⚠️ Aviso: API pediu para aguardar (Erro 429). Esperando 30 segundos...")
                    time.sleep(30)
                    continue # Tenta a mesma requisição de novo
                
                if response.status_code != 200:
                    print(f"Erro {response.status_code} ao acessar API.")
                    break

                dados = response.json()
                lista_props = dados.get("dados", [])
                links = dados.get("links", [])
                
                for prop in lista_props:
                    prop['autor_nome_busca'] = nome_deputado
                    todas_proposicoes.append(prop)
                
                link_next = next((link['href'] for link in links if link['rel'] == 'next'), None)
                
                if link_next:
                    url_atual = link_next
                    params = {}
                else:
                    break 
                
                # --- MUDANÇA 3: Delay Conservador ---
                # 1 segundo de pausa entre páginas é muito seguro.
                time.sleep(1) 
                
            except Exception as e:
                print(f"Erro de conexão: {e}")
                time.sleep(5) # Espera um pouco se a internet cair e tenta seguir
                break

print(f"\nColeta finalizada! Total: {len(todas_proposicoes)}")

if todas_proposicoes:
    df_resultado = pd.DataFrame(todas_proposicoes)
    df_resultado.to_csv("Proposicoes_2025_Seguro.csv", index=False)

Buscando: Ana Paula Lima (1/18)...
Buscando: Caroline de Toni (2/18)...
Buscando: Cobalchini (3/18)...
Buscando: Daniel Freitas (4/18)...
Buscando: Daniela Reinehr (5/18)...
Buscando: Fabio Schiochet (6/18)...
Buscando: Geovania de Sá (7/18)...
Buscando: Gilson Marques (8/18)...
Buscando: Ismael (9/18)...
Buscando: Jorge Goetten (10/18)...
Buscando: Julia Zanatta (11/18)...
Buscando: Luiz Fernando Vampiro (12/18)...
Buscando: Pedro Uczai (13/18)...
Buscando: Pezenti (14/18)...
Buscando: Ricardo Guidi (15/18)...
Buscando: Zé Trovão (16/18)...
Buscando: Carlos Chiodini (17/18)...
Buscando: Coronel Armando (18/18)...

Coleta finalizada! Total: 1752


In [7]:
len(df_resultado.id.unique())

272

In [8]:
df_resultado

,id,uri,siglaTipo,codTipo,numero,ano,ementa,dataApresentacao,autor_nome_busca
0,2485341,https://dadosabertos.camara.leg.br/api/v2/prop...,PEC,136,8,2025,"Dá nova redação ao inciso XIII, do artigo 7° d...",2025-02-25T15:47,Ana Paula Lima
1,2487818,https://dadosabertos.camara.leg.br/api/v2/prop...,PEC,136,12,2025,"Altera o art. 50 da Constituição da República,...",2025-03-19T15:25,Ana Paula Lima
2,2544926,https://dadosabertos.camara.leg.br/api/v2/prop...,PEC,136,31,2025,Altera os arts. 5º e 10 da Emenda Constitucion...,2025-08-13T15:20,Ana Paula Lima
3,2595818,https://dadosabertos.camara.leg.br/api/v2/prop...,PEC,136,48,2025,Altera o inciso XVI do art. 37 da Constituição...,2025-12-10T20:24,Ana Paula Lima
4,2596372,https://dadosabertos.camara.leg.br/api/v2/prop...,PEC,136,49,2025,Altera o § 4º do art. 220 da Constituição Fede...,2025-12-12T15:26,Ana Paula Lima
...,...,...,...,...,...,...,...,...,...
1747,2508216,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2265,2025,Revoga os arts. 359-L e 359-M do Decreto-Lei n...,2025-05-13T14:42,Coronel Armando
1748,2523716,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2792,2025,Dispõe sobre a avaliação popular dos Ministros...,2025-06-10T17:42,Coronel Armando
1749,2526321,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2914,2025,"Altera a Lei nº 10.438, de 26 de abril de 2002...",2025-06-17T10:18,Coronel Armando
1750,2533131,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3288,2025,Dispõe sobre o transporte aéreo oficial de age...,2025-07-08T17:16,Coronel Armando


In [ ]:
import pandas as pd
import requests
import time

# 1. Carregar o CSV gerado no passo anterior
arquivo_origem = "Proposicoes_2025_Seguro.csv"
try:
    df_origem = pd.read_csv(arquivo_origem)
    # Garante que temos uma lista única de IDs para não buscar duplicados
    ids_proposicoes = df_origem['id'].unique()
    print(f"Carregado: {len(ids_proposicoes)} proposições únicas para detalhar.")
except FileNotFoundError:
    print(f"Erro: O arquivo '{arquivo_origem}' não foi encontrado. Rode o script anterior primeiro.")
    exit()

# Configurações da API
url_base = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"
headers = {
    "Accept": "application/json",
    "User-Agent": "ScriptAnaliseDetalhada/1.0 (seu_email@exemplo.com)" # Mantenha seu user-agent
}

dados_detalhados = []

# Função auxiliar para fazer requisição com segurança (retry em caso de erro 429)
def requisitar_api(url):
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print(f"⏳ API ocupada (429). Aguardando 30s...")
                time.sleep(30)
                continue
            else:
                print(f"Erro {response.status_code} em: {url}")
                return None
        except Exception as e:
            print(f"Erro de conexão: {e}")
            time.sleep(5)
            return None

# 2. Loop principal (itera sobre cada ID)
total = len(ids_proposicoes)
for i, id_prop in enumerate(ids_proposicoes):
    print(f"Processando {i+1}/{total} (ID: {id_prop})...")
    
    # --- Parte A: Pegar Detalhes ---
    url_detalhes = f"{url_base}/{id_prop}"
    json_detalhes = requisitar_api(url_detalhes)
    
    if json_detalhes and 'dados' in json_detalhes:
        dado = json_detalhes['dados']
        status = dado.get('statusProposicao', {})
        
        # --- Parte B: Pegar Temas ---
        url_temas = f"{url_base}/{id_prop}/temas"
        json_temas = requisitar_api(url_temas)
        
        lista_temas_texto = []
        if json_temas and 'dados' in json_temas:
            # Transforma a lista de objetos temas em uma string única separada por ponto e vírgula
            # Ex: "Economia; Administração Pública"
            lista_temas_texto = [t['tema'] for t in json_temas['dados']]
        
        temas_concatenados = "; ".join(lista_temas_texto)

        # --- Parte C: Montar o Dicionário Final ---
        # Aqui extraímos o que é importante do JSON complexo
        registro = {
            "id": dado.get('id'),
            "uri": dado.get('uri'),
            "siglaTipo": dado.get('siglaTipo'),
            "numero": dado.get('numero'),
            "ano": dado.get('ano'),
            "ementa": dado.get('ementa'),
            "ementa_detalhada": dado.get('ementaDetalhada'),
            "keywords": dado.get('keywords'),
            "data_apresentacao": dado.get('dataApresentacao'),
            
            # Dados do Status (Objeto aninhado)
            "situacao_descricao": status.get('descricaoSituacao'), # Ex: Aguardando Despacho
            "tramitacao_descricao": status.get('descricaoTramitacao'),
            "orgao_atual": status.get('siglaOrgao'),
            "despacho": status.get('despacho'),
            "link_inteiro_teor": dado.get('urlInteiroTeor'),
            
            # Temas
            "temas": temas_concatenados
        }
        
        dados_detalhados.append(registro)
    
    # Delay de segurança entre cada proposição (essencial para não tomar ban)
    time.sleep(1)

# 3. Salvar o novo CSV
if dados_detalhados:
    df_detalhado = pd.DataFrame(dados_detalhados)
    
    # Opcional: Se quiser juntar com o nome do autor que estava no CSV original,
    # podemos fazer um merge. Mas como você pediu um "outro df", vou salvar separado.
    
    nome_arquivo_saida = "Detalhes_Proposicoes_2025.csv"
    df_detalhado.to_csv(nome_arquivo_saida, index=False)
    print(f"\n✅ Concluído! Arquivo '{nome_arquivo_saida}' gerado com sucesso.")
    print(df_detalhado.head())
else:
    print("Nenhum detalhe foi coletado.")

Carregado: 272 proposições únicas para detalhar.
Processando 1/272 (ID: 2485341)...
Processando 2/272 (ID: 2487818)...
Processando 3/272 (ID: 2544926)...
Processando 4/272 (ID: 2595818)...
Processando 5/272 (ID: 2596372)...
Processando 6/272 (ID: 2598301)...
Processando 7/272 (ID: 2483068)...
Processando 8/272 (ID: 2484047)...
Processando 9/272 (ID: 2484048)...
Processando 10/272 (ID: 2486419)...
Processando 11/272 (ID: 2486420)...
Processando 12/272 (ID: 2518417)...
Processando 13/272 (ID: 2523711)...
Processando 14/272 (ID: 2531337)...
Processando 15/272 (ID: 2536213)...
Processando 16/272 (ID: 2551371)...
Processando 17/272 (ID: 2570520)...
Processando 18/272 (ID: 2571082)...
Processando 19/272 (ID: 2578344)...
Processando 20/272 (ID: 2581571)...
Processando 21/272 (ID: 2581572)...
Processando 22/272 (ID: 2582000)...
Processando 23/272 (ID: 2582519)...
Processando 24/272 (ID: 2587740)...
Processando 25/272 (ID: 2587750)...
Processando 26/272 (ID: 2589147)...
Processando 27/272 (ID: 

In [1]:
import pandas as pd
import requests
import time

# 1. Carregar o CSV
arquivo_origem = "Proposicoes_2025_Seguro.csv"
try:
    df_origem = pd.read_csv(arquivo_origem)
    ids_proposicoes = df_origem['id'].unique()
    print(f"Carregado: {len(ids_proposicoes)} proposições para detalhar.")
except FileNotFoundError:
    print(f"Erro: Arquivo '{arquivo_origem}' não encontrado.")
    exit()

# Configurações
url_base = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"
headers = {
    "Accept": "application/json",
    "User-Agent": "ScriptAnaliseDetalhada/2.0 (seu_email@exemplo.com)"
}

dados_detalhados = []

def requisitar_api(url):
    for tentativa in range(3): # Tenta 3 vezes antes de desistir
        try:
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print(f"⏳ API cheia (429). Esperando 30s...")
                time.sleep(30)
            else:
                print(f"Erro {response.status_code} na URL: {url}")
                return None
        except Exception as e:
            print(f"Erro de conexão ({e}). Tentando novamente...")
            time.sleep(5)
    return None

# 2. Loop Principal
total = len(ids_proposicoes)
for i, id_prop in enumerate(ids_proposicoes):
    print(f"Processando {i+1}/{total} (ID: {id_prop})...")
    
    url_detalhes = f"{url_base}/{id_prop}"
    json_detalhes = requisitar_api(url_detalhes)
    
    if json_detalhes and 'dados' in json_detalhes:
        dado = json_detalhes['dados']
        
        # --- CORREÇÃO 1: Tratamento de Nulo Explícito ---
        # O 'or {}' garante que se vier None, vira um dicionário vazio e não quebra
        status = dado.get('statusProposicao') or {}
        
        # --- CORREÇÃO 2: Garantia de Dados (Fallback) ---
        situacao_api = status.get('descricaoSituacao')
        tramitacao_api = status.get('descricaoTramitacao')
        
        # Lógica: Se a situação vier vazia (None ou ""), usa a tramitação como "tapa-buraco"
        if situacao_api:
            situacao_final = situacao_api
        elif tramitacao_api:
             # Marca com asterisco para você saber que foi preenchido automaticamente
            situacao_final = f"{tramitacao_api} (Status Inferido)"
        else:
            situacao_final = "Sem informação"

        # Pega Temas
        url_temas = f"{url_base}/{id_prop}/temas"
        json_temas = requisitar_api(url_temas)
        temas_concatenados = ""
        if json_temas and 'dados' in json_temas:
            lista_temas = [t['tema'] for t in json_temas['dados']]
            temas_concatenados = "; ".join(lista_temas)

        registro = {
            "id": dado.get('id'),
            "uri": dado.get('uri'),
            "siglaTipo": dado.get('siglaTipo'),
            "numero": dado.get('numero'),
            "ano": dado.get('ano'),
            "ementa": dado.get('ementa'),
            
            # Campos corrigidos
            "situacao_descricao": situacao_final, 
            "tramitacao_descricao": tramitacao_api,
            
            "orgao_atual": status.get('siglaOrgao'),
            "despacho": status.get('despacho'),
            "link_inteiro_teor": dado.get('urlInteiroTeor'),
            "temas": temas_concatenados
        }
        
        dados_detalhados.append(registro)
    
    time.sleep(0.5) 

# 3. Salvar
if dados_detalhados:
    df_detalhado = pd.DataFrame(dados_detalhados)
    df_detalhado.to_csv("Detalhes_Proposicoes_Corrigido.csv", index=False)
    print("\n✅ Concluído! Arquivo 'Detalhes_Proposicoes_Corrigido.csv' salvo.")
    
    # Verifica o ID problemático
    check = df_detalhado[df_detalhado['id'] == 2570520]
    if not check.empty:
        print("\nVerificação do ID 2570520:")
        print(check[['id', 'situacao_descricao', 'tramitacao_descricao']].to_string())
else:
    print("Nenhum dado coletado.")

Carregado: 272 proposições para detalhar.
Processando 1/272 (ID: 2485341)...
Processando 2/272 (ID: 2487818)...
Processando 3/272 (ID: 2544926)...
Processando 4/272 (ID: 2595818)...
Processando 5/272 (ID: 2596372)...
Processando 6/272 (ID: 2598301)...
Processando 7/272 (ID: 2483068)...
Processando 8/272 (ID: 2484047)...
Processando 9/272 (ID: 2484048)...
Processando 10/272 (ID: 2486419)...
Processando 11/272 (ID: 2486420)...
Processando 12/272 (ID: 2518417)...
Processando 13/272 (ID: 2523711)...
Processando 14/272 (ID: 2531337)...
Processando 15/272 (ID: 2536213)...
Processando 16/272 (ID: 2551371)...
Processando 17/272 (ID: 2570520)...
Processando 18/272 (ID: 2571082)...
Processando 19/272 (ID: 2578344)...
Processando 20/272 (ID: 2581571)...
Processando 21/272 (ID: 2581572)...
Processando 22/272 (ID: 2582000)...
Processando 23/272 (ID: 2582519)...
Processando 24/272 (ID: 2587740)...
Processando 25/272 (ID: 2587750)...
Processando 26/272 (ID: 2589147)...
Processando 27/272 (ID: 2589158

### Como os dados podem vir duplicados no excel eu removi duplicadas de acordo com todas as linhas, foi removido 438 linhas